In [1]:
import os

os.chdir("../")
%pwd

'/home/siddhu/Desktop/Text-Summarizer-With-HF'

In [2]:
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [3]:
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [5]:
class DataTransformation:
    
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.tokenizer_name
        )

    def convert_examples_to_features(self, example_batch):
        model_inputs = self.tokenizer(
            example_batch["dialogue"],  # Tokenize input dialogue
            text_target=example_batch["summary"],  # Tokenize target summary for seq2seq
            max_length=1024,  # Max length for input tokens
            truncation=True,  # Truncate if longer than max_length
        )
        labels = self.tokenizer(
            text_target=example_batch["summary"],
            max_length=128,
            truncation=True,  # Tokenize summary as labels
        )
        model_inputs["labels"] = labels["input_ids"]  # Add tokenized labels to model inputs
        return model_inputs

    def convert(self):
        dataset_splits = {}

        for split in ["train", "validation", "test"]:
            csv_path = os.path.join(
                self.config.data_path, f"{split}.csv"
            )

            df = pd.read_csv(csv_path)
            ds = Dataset.from_pandas(df)

            ds = ds.map(
                self.convert_examples_to_features,
                batched=True
            )

            dataset_splits[split] = ds

        summarizer_dataset = DatasetDict(dataset_splits)

        summarizer_dataset.save_to_disk(
            os.path.join(self.config.root_dir, "summarizer_dataset")
        )

In [6]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2026-02-01 09:59:42,404: INFO: common: YAML file 'config/config.yaml' read successfully.]
[2026-02-01 09:59:42,406: INFO: common: YAML file 'params.yaml' read successfully.]
[2026-02-01 09:59:42,407: INFO: common: Directory 'artifacts' created successfully or already exists.]
[2026-02-01 09:59:42,408: INFO: common: Directory 'artifacts/data_transformation' created successfully or already exists.]


Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14731 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]